## Documentation

- API Guide can be found in this link https://ib-insync.readthedocs.io/api.html
- Options Example (SPX) with ib_insync https://nbviewer.org/github/erdewit/ib_insync/blob/master/notebooks/option_chain.ipynb


In [33]:
from ib_insync import *
import pandas as pd
util.startLoop()
from datetime import date
from datetime import datetime

def verify_daycount(dates):
    ds = []
    today = date.today()
    today = datetime.strftime(today, "%Y%m%d")
    today = datetime.strptime(today, "%Y%m%d")
    for adate in dates:
        date_obj = datetime.strptime(adate, "%Y%m%d")
        days_diff = (date_obj - today).days
        if (34 < days_diff < 57) or (90 < days_diff < 150):
            ds.append(adate)
    return ds
    

ib = IB()
ib.connect('127.0.0.1', 7496, clientId = 1)

tick = 'AAPL'
print( " >>> DEFINING STOCK ")
stk = Stock(tick, 'SMART', 'USD')
ib.qualifyContracts(stk)
ib.reqMarketDataType(4) #wait a bit
chains = ib.reqSecDefOptParams(stk.symbol, '', stk.secType, stk.conId)
ds_chains = pd.DataFrame(chains)
expirations = sorted(ds_chains['expirations'].iloc[0])
expirations = verify_daycount(expirations) #convert expiration day to number of days left to expire
strikes = ds_chains['strikes'].iloc[0]
contractID = ds_chains['underlyingConId'].iloc[0]
rights = ['P', 'C'] #easier for algirthm built # P = Put ; C = Call

#organize all Option Contracts into data set
print( " >>> DEFINING OPTIONS ")
contracts = [Option(tick, ex, s, r, 'SMART') for r in rights for ex in expirations for s in strikes]
print( " >>> QUALIFYING OPTIONS ")
qualifiedContracts = ib.qualifyContracts(*contracts)
print( " >>> RETRIEVING DETAILS ")
qualifiedContractDetails = ib.reqTickers(*qualifiedContracts)
ds = pd.DataFrame()
print ( " >>> GETTING GREEKS AND OPTIONS INFO")
for qcd in qualifiedContractDetails:
    try:
        details = qcd.contract
        exp, strike, right = details.lastTradeDateOrContractMonth, details.strike, details.right
        greek = qcd.modelGreeks
        delta, theta, gamma, vega, pvDiv, optPrice, impliedVol = greek.delta, greek.theta, greek.gamma, greek.vega, greek.pvDividend, greek.optPrice, greek.impliedVol
        row = {
            'Symbol': tick, 'Expiration': exp , 'Strike': strike, 'Right': right,
            'Delta': delta, 'Theta': theta, 'Vega': vega, 'Gamma': gamma,
            'impVol': impliedVol, 'optPrice': optPrice, 'Ask': qcd.ask, 'Bid': qcd.bid, 'Last': qcd.last}
        ds = pd.concat([ds, pd.DataFrame([row])], axis = 0, ignore_index = True)
    except:
        print ( " >>> NOGOOD CONTRACT")

#next: call all options during market time to get greek values
def days_diff(adate):
    today = date.today()
    today = datetime.strftime(today, "%Y%m%d")
    today = datetime.strptime(today, "%Y%m%d")
    date_obj = datetime.strptime(adate, "%Y%m%d")
    return (date_obj - today).days

ds['daysdiff'] = list(map(days_diff, ds['Expiration'])) 
ds['Delta'] = abs(ds['Delta'])
ds['Theta'] = abs(ds['Theta'])
print(ds)

# ds_p1 = ds[(0.1 <= ds['Delta'] <= 0.2) & (ds['Right'] == 'P') & (days_diff(ds['Expiration'], 90, 150))]
ds_p1 = ds[(ds['Right'] == 'P') & (ds['daysdiff'] >= 90) & (ds['daysdiff'] <= 150) & (ds['Delta'] >= 0.1) & (ds['Delta'] <= 0.2)]
ds_p1['REGEL'] = ['P1'] * len(ds_p1)
ds_p2 = ds[(ds['Right'] == 'P') & (ds['daysdiff'] >= 34) & (ds['daysdiff'] <= 57) & (ds['Delta'] >= 0.2) & (ds['Delta'] <= 0.3)]
ds_p2['REGEL'] = ['P2'] * len(ds_p2)
ds_c3 = ds[(ds['Right'] == 'C') & (ds['daysdiff'] >= 34) & (ds['daysdiff'] <= 57) & (ds['Delta'] >= 0.2) & (ds['Delta'] <= 0.3)]
ds_c3['REGEL'] = ['C3'] * len(ds_c3)
ds_c4 = ds[(ds['Right'] == 'C') & (ds['daysdiff'] >= 90) & (ds['daysdiff'] <= 150) & (ds['Delta'] >= 0.1) & (ds['Delta'] <= 0.2)]
ds_c4['REGEL'] = ['C4'] * len(ds_c4)

main = pd.concat([ds_p1, ds_p2, ds_c3, ds_c4], ignore_index=True)
def p_marg(p1, p2):
    if (p1 == p2 - 10) | (p1 == p2 - 20):
        return True
    return False

def c_marg(c3, c4):
    if (c4 == c3 + 10) | (c4 == c3 + 20):
        return True
    return False

def p_theta(p1, p2):
    if (p2 > p1 * 1.5):
        return True
    return False

def c_theta(c3, c4):
    if (c3 > c4 * 1.5):
        return True
    return False

def bundle(ds):
    p1 = ds[ds['REGEL'] == 'P1']
    p2 = ds[ds['REGEL'] == 'P2']
    c3 = ds[ds['REGEL'] == 'C3']
    c4 = ds[ds['REGEL'] == 'C4']
    ps = []
    cs = []
    i = 0

    #convert two loops to function prperly (rearrange functions in sheet)
    for i in range(len(p1)):
        j = 0
        for j in range(len(p2)):
            # code begin
            if p_marg(p1.iloc[i]['Strike'], p2.iloc[j]['Strike']) & p_theta(p1.iloc[i]['Theta'], p2.iloc[j]['Theta']):
                ps.append((i, j))
            # code end
            j += 1
        i += 1

    for i in range(len(c3)):
        j = 0
        for j in range(len(c4)):
            # code begin
            if c_marg(c3.iloc[i]['Strike'], c4.iloc[j]['Strike']) & c_theta(c3.iloc[i]['Theta'], c4.iloc[j]['Theta']):
                cs.append((i, j))
            # code end
            j += 1
        i += 1
    

    legs_indx = [(p1, p2, c3, c4) for (p1, p2) in ps for (c3, c4) in cs]

    dss = []
    for (a, b, c, d) in legs_indx:
        ds = pd.concat([p1.iloc[a].to_frame().T, p2.iloc[b].to_frame().T, c3.iloc[c].to_frame().T, c4.iloc[d].to_frame().T], ignore_index=True)
        # df = pd.DataFrame.from_records(ds)
        dss.append(ds)
    
    return dss


bundles = bundle(main)

pd.concat(bundles).to_csv(tick+'_options.csv')




# Create the main line at each bundle sheet
contract = Stock('AAPL','SMART','USD')
# ib = IB().connect('127.0.0.1', 4001 , clientId=100) #connect
m_data = ib.reqMktData(contract)
while m_data.last != m_data.last: ib.sleep(0.01) #Wait until data is in. 
ib.cancelMktData(contract)
last_price = m_data.last

stock = Stock(tick, 'SMART', 'USD') #, primaryExchange='NASDAQ'
bars = ib.reqHistoricalData(stock, endDateTime = '', durationStr = '26 W',barSizeSetting = '1 hour', whatToShow = 'MIDPOINT', useRTH = True)
lows, highs = [], []

for bar in bars:
    lows.append(bar.low)
    highs.append(bar.high)

bars = ib.reqHistoricalData(stock, endDateTime = '', durationStr = '2 D',barSizeSetting = '1 day', whatToShow = 'MIDPOINT', useRTH = True)
change = ((bars[0].close - last_price) / last_price) * 100


ds__ = pd.DataFrame({'lows': lows,'highs': highs})
ds['Delta'] = ds['Delta'].apply(lambda x: round(x, 2))
mini = ds__['lows'].apply(lambda x: round(x, 0)).min()
maxi = ds__['highs'].apply(lambda x: round(x, 0)).max()
ib.disconnect()

d = " - "
mids = { #main info dataset
'Symbol': tick, 
'Last': last_price,
'Change%': round(change, 2),
'Low-26W': round(mini, 2),
'High-26W': round(maxi, 2),
"IV%": d,
"H-IV-30%": d,
"H-IV-Mean%": 23,
"RSI-14D": d
}

mids = pd.DataFrame([mids])
print(mids)








import openpyxl
import xlsxwriter
import numpy as np
# bundles.to_excel('pandas_to_excel.xlsx', sheet_name='new_sheet_name')

writer = pd.ExcelWriter('bundles.xlsx',engine='xlsxwriter')
workbook = writer.book
worksheet = workbook.add_worksheet('Result')
writer.sheets['Bundles'] = worksheet

mids.to_excel(writer, sheet_name = 'Bundles', startrow = 0, startcol = 0)


bundle_count = 1
currow, curcol = 4, 0
for ds in bundles:
        # ----------------- OPTION DF -------------------------
        symbol = str(list(ds[ds['REGEL'] == 'P1']['Symbol'])[0])
        # worksheet.write_string(currow, 0, str(symbol) + " " + str(bundle_count) + "/" + str(len(bundles)))
        ds['Change'] = ((ds['Strike'] - last_price)/last_price) #convert to percentage
        ds['Change'] = ds['Change'].apply(lambda x: round(x, 2))
        ds['Mark'] = (ds['Ask'] + ds['Bid']) / 2
        ds.loc[ds.REGEL == 'P2', 'Mark'] *= -1
        ds.loc[ds.REGEL == 'C3', 'Mark'] *= -1
        ds['Mark'] = ds['Mark'].apply(lambda x: round(x, 2))
        ds = ds.rename({'impVol': 'IV-OP'}, axis = 1)
        ds['IV-OP'] = ds['IV-OP'].apply(lambda x: round(x, 2))
        # ds['Expiration'] = ds['Expiration'].dt.strftime('%y-%m-%d')
        ds['Expiration'] = pd.to_datetime(ds.Expiration, format='%Y%m%d')
        ds['Expiration'] = ds['Expiration'].dt.strftime('%y-%m-%d')
        ds = ds.rename({'daysdiff': 'Days'}, axis = 1) 
        ds['Delta'] = ds['Delta'].apply(lambda x: round(x, 2))
        ds['Theta'] = ds['Theta'].apply(lambda x: round(x, 2))
        ds['Vega'] = ds['Vega'].apply(lambda x: round(x, 2))


        mp, mc = list(ds[ds['REGEL'] == 'P2']['Strike'])[0] - list(ds[ds['REGEL'] == 'P1']['Strike'])[0], list(ds[ds['REGEL'] == 'C4']['Strike'])[0] - list(ds[ds['REGEL'] == 'C3']['Strike'])[0]
        vega_p1, vega_p2, vega_c3, vega_c4 = list(ds[ds['REGEL'] == 'P1']['Vega'])[0], list(ds[ds['REGEL'] == 'P2']['Vega'])[0], list(ds[ds['REGEL'] == 'C3']['Vega'])[0], list(ds[ds['REGEL'] == 'C4']['Vega'])[0]
        delta_p1, delta_p2, delta_c3, delta_c4 = list(ds[ds['REGEL'] == 'P1']['Delta'])[0], list(ds[ds['REGEL'] == 'P2']['Delta'])[0], list(ds[ds['REGEL'] == 'C3']['Delta'])[0], list(ds[ds['REGEL'] == 'C4']['Delta'])[0]
        days_p1, days_p2, days_c3, days_c4 = list(ds[ds['REGEL'] == 'P1']['Days'])[0], list(ds[ds['REGEL'] == 'P2']['Days'])[0], list(ds[ds['REGEL'] == 'C3']['Days'])[0], list(ds[ds['REGEL'] == 'C4']['Days'])[0]
        iv_p1, iv_p2, iv_c3, iv_c4 = list(ds[ds['REGEL'] == 'P1']['IV-OP'])[0], list(ds[ds['REGEL'] == 'P2']['IV-OP'])[0], list(ds[ds['REGEL'] == 'C3']['IV-OP'])[0], list(ds[ds['REGEL'] == 'C4']['IV-OP'])[0]
        mark_p1, mark_p2, mark_c3, mark_c4 = list(ds[ds['REGEL'] == 'P1']['Mark'])[0], list(ds[ds['REGEL'] == 'P2']['Mark'])[0], list(ds[ds['REGEL'] == 'C3']['Mark'])[0], list(ds[ds['REGEL'] == 'C4']['Mark'])[0]


        rtd_p, rtd_c = list(ds[ds['REGEL'] == 'P2']['Theta'])[0] - list(ds[ds['REGEL'] == 'P1']['Theta'])[0], list(ds[ds['REGEL'] == 'C3']['Theta'])[0] - list(ds[ds['REGEL'] == 'C4']['Theta'])[0]
        ds['RTD'] = [rtd_p, rtd_p, rtd_c, rtd_c]
        ds['RTM'] = ds['RTD'] * 22
        ds['RTM'] = ds['RTM'].apply(lambda x: round(x, 2))
        ds['RTY'] = ds['RTM'] * 12
        ds['Margin'] = [mp, mp, mc, mc]


        title = str(symbol) + " " + str(bundle_count) + "/" + str(len(bundles))
        ds = ds.rename({'REGEL': title}, axis = 1)
        ds.set_index(title, inplace = True)
        ds[['Expiration', 'Days', 'Strike', 'Change', 'Mark', 'Delta', 'Theta', 'Vega', 'IV-OP', 'RTM']].to_excel(writer, sheet_name = 'Bundles', startrow = currow, startcol = 0)
        currow += ds.shape[0] + 2

        # ----------------- NATIVE DF -------------------------
        mc = mc.astype(np.int64)
        mp = mp.astype(np.int64)
        
        if mc > mp:
                max_margin = mc
        else:
                max_margin = mp

        pnative = {
        'Type': "P-Native", 
        'Margin': mp,
        'Monthly%': (rtd_p * 22 * 100) / mp,
        'Yearly%': (rtd_p * 22 * 12 * 100) / mp,
        'Yearly$': rtd_p * 22 * 12,
        "Cost": list(ds[ds.index == 'P1']['Mark'])[0] + list(ds[ds.index == 'P2']['Mark'])[0]
        }

        cnative = {
        'Type': "C-Native", 
        'Margin': mc,
        'Monthly%': (rtd_c * 22 * 100) / mc,
        'Yearly%': (rtd_c * 22 * 12 * 100) / mc,
        'Yearly$': (rtd_c * 22 * 12),
        "Cost": list(ds[ds.index == 'C3']['Mark'])[0] + list(ds[ds.index == 'C4']['Mark'])[0]
        }

        posnative = {
        'Type': "POS-Native", 
        'Margin': max_margin,
        'Monthly%': ((rtd_c * 22 + rtd_p * 22) / max_margin) * 100,
        'Yearly%': (((rtd_c * 22 * 12) + (rtd_p * 22 * 12)) / max_margin) * 100,
        'Yearly$': (pnative['Yearly$'] + cnative['Yearly$']),
        "Cost": ds['Mark'].sum()
        }

        ds_ = pd.DataFrame([pnative, cnative, posnative])
        ds_.set_index('Type', inplace = True)
        ds_.to_excel(writer, sheet_name = 'Bundles', startrow = currow, startcol = 0)
        currow += 5 # 4 for the native ds_, 2 for spacing

        # ----------------- GRADES DF -------------------------
        def calculate_grade(value, graph_points):
            keys, values = list(graph_points.keys()), list(graph_points.values())
            i = 0
            for i in range(len(keys) - 1):
                if (values[i] < value) & (value < values[i+1]):
                    m = (values[i] - values[i + 1])/(keys[i] - keys[i + 1])
                    b = values[i]
                    return  round(m * value - b, 2) # y = m * x + b

            i += 1
            return 0



        # ----------------- OPTIMUM DF -------------------------

        optimum_value = {
        'OPTIMUM': "Value", 
        'Total': " - ", # TODO 
        'Yearly%': (((rtd_c * 22 * 12) + (rtd_p * 22 * 12)) / max_margin) * 100,
        'Delta Ratio': 0, # TODO 
        'Delta Neutral': delta_p1 + delta_p2 + delta_c3 + delta_c4,
        'Vega Neutral': (vega_p2 + vega_c3) - (vega_p1 + vega_c4),
        'Day P1': days_p1,
        'Day P2': days_p2,
        'Day C3': days_c3,
        'Day C4': days_c4,
        'Stk Dist Mid 26W': round((mini + maxi) / 2, 2),
        'IV Gap Stk': 0, # TODO 
        'IV P/C': round(iv_p2 / iv_c3, 2),
        'P Margin': mp,
        'C Margin': mc,
        'Max Margin': max_margin,
        'RSI': last_price
        }

        optimum_grade = {
        'OPTIMUM': "Grade", 
        'Total': 0,
        'Yearly%': 0,
        'Delta Ratio': 0,
        'Delta Neutral': 0,
        'Vega Neutral': 0,
        'Day P1': calculate_grade(optimum_value['Day P1'], {60: 0, 128: 100}),
        'Day P2': calculate_grade(optimum_value['Day P2'], {34:10, 45:100, 57:10}),
        'Day C3': calculate_grade(optimum_value['Day C3'], {34:10, 45:100, 57:10}),
        'Day C4': calculate_grade(optimum_value['Day C4'], {60: 0, 128: 100}),
        'Stk Dist Mid 26W': calculate_grade(optimum_value['Stk Dist Mid 26W'], {(optimum_value['Stk Dist Mid 26W'] - 15):10, optimum_value['Stk Dist Mid 26W']:100, (optimum_value['Stk Dist Mid 26W'] + 30):10}),
        'IV Gap Stk': 0, # TODO 
        'IV P/C': calculate_grade(optimum_value['IV P/C'], {0.8:0, 1.1:100, 1.4:0}),
        'P Margin': calculate_grade(optimum_value['P Margin'], {5:100, 10:60, 15:0}),
        'C Margin': calculate_grade(optimum_value['C Margin'], {10:100, 15:75, 20:0}),
        'Max Margin': calculate_grade(optimum_value['Max Margin'], {10:100, 15:75, 20:0}),
        'RSI': calculate_grade(optimum_value['RSI'], {30:0, 40:100, 50:100, 60:0})
        }

        optimum_weight = {
        'OPTIMUM': "Weight", 
        'Total': 0,
        'Yearly%': 0,
        'Delta Ratio': 0,
        'Delta Neutral': 0,
        'Vega Neutral': 0,
        'Day P1': optimum_grade['Day P1'] * 5,
        'Day P2': optimum_grade['Day P2'] * 5,
        'Day C3': optimum_grade['Day C3'] * 5,
        'Day C4': optimum_grade['Day C4'] * 5,
        'Stk Dist Mid 26W': optimum_grade['Stk Dist Mid 26W'] * 5,
        'IV Gap Stk': 0,
        'IV P/C': optimum_grade['IV P/C'] * 5,
        'P Margin': optimum_grade['P Margin'] * 5,
        'C Margin': optimum_grade['C Margin'] * 5,
        'Max Margin': optimum_grade['Max Margin'] * 10,
        'RSI': optimum_grade['RSI'] * 10
        }

        ds_optimum = pd.DataFrame([optimum_value, optimum_grade, optimum_weight])
        ds_optimum.set_index('OPTIMUM', inplace = True)
        ds_optimum.to_excel(writer, sheet_name = 'Bundles', startrow = currow, startcol = 0)
        currow += 5 # 4 for the ds_
        # ----------------- PROFIT DF -------------------------
        profit_value = {
        'PROFIT': "Value", 
        'Total': " - ", # TODO 
        'Cost-P': 0, #round(mark_p1 / (mark_p1 + mark_p2), 2),
        'Cost-C': 0, #round(mark_c4 / (mark_c3 + mark_c4), 2),
        'Cost-P/C': round((mark_p1 + mark_c4) / (mark_p1 + mark_p2 + mark_c3 + mark_c4), 2),
        'Theta-P1/P2': 0,
        'Theta-C3/C4': 0,
        'RTM-P': round(rtd_p * 22, 2),
        'RTM-C': round(rtd_c * 22, 2),
        'IV-P-Gap': round(iv_p2 - iv_p1 ,2),
        'IV-C-Gap': round(iv_c3 - iv_c4 ,2)
        }

        profit_value = {
        'PROFIT': "Value", 
        'Total': " - ", # TODO 
        'Cost-P': calculate_grade(optimum_value['Cost-P'], {0.8:0, 1.1:100, 1.4:0}),
        'Cost-C': 0,
        'Cost-P/C': 0,
        'Theta-P1/P2': 0,
        'Theta-C3/C4': 0,
        'RTM-P': 0,
        'RTM-C': 0,
        'IV-P-Gap': 0,
        'IV-C-Gap': 0
        }
                

        profit_weight = {
        'PROFIT': "Value", 
        'Total': " - ", # TODO 
        'Cost-P': 0,
        'Cost-C': 0,
        'Cost-P/C': 0,
        'Theta-P1/P2': 0,
        'Theta-C3/C4': 0,
        'RTM-P': 0,
        'RTM-C': 0,
        'IV-P-Gap': 0,
        'IV-C-Gap': 0
        }            

        ds_profit = pd.DataFrame([optimum_value, optimum_grade, optimum_weight])
        ds_profit.set_index('PROFIT', inplace = True)
        ds_profit.to_excel(writer, sheet_name = 'Bundles', startrow = currow, startcol = 0)
        currow += 5 # 4 for the ds_


        # ----------------- RISK DF -------------------------
        currow += 7 # 4 for the ds_, 3 for spacing
        bundle_count += 1

writer.save() #save the excelt sheet



ib.disconnect()


 >>> DEFINING STOCK 


Error 200, reqId 5: No security definition has been found for the request, contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20220902', strike=30.0, right='P', exchange='SMART')
Error 200, reqId 6: No security definition has been found for the request, contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20220902', strike=35.0, right='P', exchange='SMART')
Error 200, reqId 7: No security definition has been found for the request, contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20220902', strike=40.0, right='P', exchange='SMART')
Error 200, reqId 8: No security definition has been found for the request, contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20220902', strike=41.25, right='P', exchange='SMART')
Error 200, reqId 10: No security definition has been found for the request, contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20220902', strike=43.75, right='P', exchange='SMART')
Error 200, reqId 9: No security definition has 

 >>> DEFINING OPTIONS 
 >>> QUALIFYING OPTIONS 


Error 200, reqId 41: No security definition has been found for the request, contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20220902', strike=102.5, right='P', exchange='SMART')
Error 200, reqId 43: No security definition has been found for the request, contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20220902', strike=107.5, right='P', exchange='SMART')
Error 200, reqId 45: No security definition has been found for the request, contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20220902', strike=112.5, right='P', exchange='SMART')
Error 200, reqId 47: No security definition has been found for the request, contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20220902', strike=117.5, right='P', exchange='SMART')
Error 200, reqId 48: No security definition has been found for the request, contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20220902', strike=119.0, right='P', exchange='SMART')
Error 200, reqId 50: No security definit

KeyboardInterrupt: 

In [7]:
ib.disconnect()

In [1]:
import openpyxl
import xlsxwriter
import numpy as np
import math

# bundles.to_excel('pandas_to_excel.xlsx', sheet_name='new_sheet_name')

writer = pd.ExcelWriter('bundles.xlsx',engine='xlsxwriter')
workbook = writer.book
worksheet = workbook.add_worksheet('Result')
writer.sheets['Bundles'] = worksheet

mids.to_excel(writer, sheet_name = 'Bundles', startrow = 0, startcol = 0)


bundle_count = 1
currow, curcol = 4, 0
for ds in bundles:
        # ----------------- OPTION DF -------------------------
        symbol = str(list(ds[ds['REGEL'] == 'P1']['Symbol'])[0])
        # worksheet.write_string(currow, 0, str(symbol) + " " + str(bundle_count) + "/" + str(len(bundles)))
        ds['Change'] = ((ds['Strike'] - last_price)/last_price) #convert to percentage
        ds['Change'] = ds['Change'].apply(lambda x: round(x, 2))
        ds['Mark'] = (ds['Ask'] + ds['Bid']) / 2
        ds.loc[ds.REGEL == 'P2', 'Mark'] *= -1
        ds.loc[ds.REGEL == 'C3', 'Mark'] *= -1
        ds['Mark'] = ds['Mark'].apply(lambda x: round(x, 2))
        ds = ds.rename({'impVol': 'IV-OP'}, axis = 1)
        ds['IV-OP'] = ds['IV-OP'].apply(lambda x: round(x, 2))
        # ds['Expiration'] = ds['Expiration'].dt.strftime('%y-%m-%d')
        ds['Expiration'] = pd.to_datetime(ds.Expiration, format='%Y%m%d')
        ds['Expiration'] = ds['Expiration'].dt.strftime('%y-%m-%d')
        ds = ds.rename({'daysdiff': 'Days'}, axis = 1) 
        ds['Delta'] = ds['Delta'].apply(lambda x: round(x, 2))
        ds['Theta'] = ds['Theta'].apply(lambda x: round(x, 2))
        ds['Vega'] = ds['Vega'].apply(lambda x: round(x, 2))


        mp, mc = list(ds[ds['REGEL'] == 'P2']['Strike'])[0] - list(ds[ds['REGEL'] == 'P1']['Strike'])[0], list(ds[ds['REGEL'] == 'C4']['Strike'])[0] - list(ds[ds['REGEL'] == 'C3']['Strike'])[0]
        vega_p1, vega_p2, vega_c3, vega_c4 = list(ds[ds['REGEL'] == 'P1']['Vega'])[0], list(ds[ds['REGEL'] == 'P2']['Vega'])[0], list(ds[ds['REGEL'] == 'C3']['Vega'])[0], list(ds[ds['REGEL'] == 'C4']['Vega'])[0]
        delta_p1, delta_p2, delta_c3, delta_c4 = list(ds[ds['REGEL'] == 'P1']['Delta'])[0], list(ds[ds['REGEL'] == 'P2']['Delta'])[0], list(ds[ds['REGEL'] == 'C3']['Delta'])[0], list(ds[ds['REGEL'] == 'C4']['Delta'])[0]
        days_p1, days_p2, days_c3, days_c4 = list(ds[ds['REGEL'] == 'P1']['Days'])[0], list(ds[ds['REGEL'] == 'P2']['Days'])[0], list(ds[ds['REGEL'] == 'C3']['Days'])[0], list(ds[ds['REGEL'] == 'C4']['Days'])[0]
        iv_p1, iv_p2, iv_c3, iv_c4 = list(ds[ds['REGEL'] == 'P1']['IV-OP'])[0], list(ds[ds['REGEL'] == 'P2']['IV-OP'])[0], list(ds[ds['REGEL'] == 'C3']['IV-OP'])[0], list(ds[ds['REGEL'] == 'C4']['IV-OP'])[0]
        mark_p1, mark_p2, mark_c3, mark_c4 = list(ds[ds['REGEL'] == 'P1']['Mark'])[0], list(ds[ds['REGEL'] == 'P2']['Mark'])[0], list(ds[ds['REGEL'] == 'C3']['Mark'])[0], list(ds[ds['REGEL'] == 'C4']['Mark'])[0]
        strike_p1, strike_p2, strike_c3, strike_c4 = list(ds[ds['REGEL'] == 'P1']['Strike'])[0], list(ds[ds['REGEL'] == 'P2']['Strike'])[0], list(ds[ds['REGEL'] == 'C3']['Strike'])[0], list(ds[ds['REGEL'] == 'C4']['Strike'])[0]


        rtd_p, rtd_c = list(ds[ds['REGEL'] == 'P2']['Theta'])[0] - list(ds[ds['REGEL'] == 'P1']['Theta'])[0], list(ds[ds['REGEL'] == 'C3']['Theta'])[0] - list(ds[ds['REGEL'] == 'C4']['Theta'])[0]
        ds['RTD'] = [rtd_p, rtd_p, rtd_c, rtd_c]
        ds['RTM'] = ds['RTD'] * 22
        ds['RTM'] = ds['RTM'].apply(lambda x: round(x, 2))
        ds['RTY'] = ds['RTM'] * 12
        ds['Margin'] = [mp, mp, mc, mc]


        title = str(symbol) + " " + str(bundle_count) + "/" + str(len(bundles))
        ds = ds.rename({'REGEL': title}, axis = 1)
        ds.set_index(title, inplace = True)
        ds[['Expiration', 'Days', 'Strike', 'Change', 'Mark', 'Delta', 'Theta', 'Vega', 'IV-OP', 'RTM']].to_excel(writer, sheet_name = 'Bundles', startrow = currow, startcol = 0)
        currow += ds.shape[0] + 2

        # ----------------- NATIVE DF -------------------------
        mc = mc.astype(np.int64)
        mp = mp.astype(np.int64)
        
        if mc > mp:
                max_margin = mc
        else:
                max_margin = mp

        pnative = {
        'Type': "P-Native", 
        'Margin': mp,
        'Monthly%': (rtd_p * 22 * 100) / mp,
        'Yearly%': (rtd_p * 22 * 12 * 100) / mp,
        'Yearly$': rtd_p * 22 * 12,
        "Cost": list(ds[ds.index == 'P1']['Mark'])[0] + list(ds[ds.index == 'P2']['Mark'])[0]
        }

        cnative = {
        'Type': "C-Native", 
        'Margin': mc,
        'Monthly%': (rtd_c * 22 * 100) / mc,
        'Yearly%': (rtd_c * 22 * 12 * 100) / mc,
        'Yearly$': (rtd_c * 22 * 12),
        "Cost": list(ds[ds.index == 'C3']['Mark'])[0] + list(ds[ds.index == 'C4']['Mark'])[0]
        }

        posnative = {
        'Type': "POS-Native", 
        'Margin': max_margin,
        'Monthly%': ((rtd_c * 22 + rtd_p * 22) / max_margin) * 100,
        'Yearly%': (((rtd_c * 22 * 12) + (rtd_p * 22 * 12)) / max_margin) * 100,
        'Yearly$': (pnative['Yearly$'] + cnative['Yearly$']),
        "Cost": ds['Mark'].sum()
        }

        ds_ = pd.DataFrame([pnative, cnative, posnative])
        ds_.set_index('Type', inplace = True)
        ds_.to_excel(writer, sheet_name = 'Bundles', startrow = currow, startcol = 0)
        currow += 5 # 4 for the native ds_, 2 for spacing

        # ----------------- GRADES DF -------------------------
        def calculate_grade(value, graph_points): # TODO: Work on algoirthm
            keys, values = list(graph_points.keys()), list(graph_points.values())
            print(keys, values)
            i = 0
            for i in range(len(keys) - 1):
                if (keys[i] < value) & (value < keys[i+1]):
                    x2, x1 = keys[i + 1] - keys[i], 0
                    y2, y1 = values[i + 1], values[i]
                    b = values[i]
                    m = (y2 - y1)/(x2 - x1)
                    return  round(m * value + b, 2) # y = m * x + b

            i += 1
            return -1



        # ----------------- OPTIMUM DF -------------------------

        optimum_value = {
        'OPTIMUM': "Value", 
        'Total': " - ", # TODO 
        'Yearly%': (((rtd_c * 22 * 12) + (rtd_p * 22 * 12)) / max_margin) * 100,
        'Delta Ratio': 0, # TODO 
        'Delta Neutral': delta_p1 + delta_p2 + delta_c3 + delta_c4,
        'Vega Neutral': (vega_p2 + vega_c3) - (vega_p1 + vega_c4),
        'Day P1': days_p1,
        'Day P2': days_p2,
        'Day C3': days_c3,
        'Day C4': days_c4,
        'Stk Dist Mid 26W': round((mini + maxi) / 2, 2),
        'IV Gap Stk': 0, # TODO 
        'IV P/C': round(iv_p2 / iv_c3, 2),
        'P Margin': mp,
        'C Margin': mc,
        'Max Margin': max_margin,
        'RSI': last_price
        }

        optimum_grade = {
        'OPTIMUM': "Grade", 
        'Total': 0,
        'Yearly%': 0,
        'Delta Ratio': 0,
        'Delta Neutral': 0,
        'Vega Neutral': 0,
        'Day P1': calculate_grade(optimum_value['Day P1'], {60: 0, 128: 100}),
        'Day P2': calculate_grade(optimum_value['Day P2'], {34:10, 45:100, 57:10}),
        'Day C3': calculate_grade(optimum_value['Day C3'], {34:10, 45:100, 57:10}),
        'Day C4': calculate_grade(optimum_value['Day C4'], {60: 0, 128: 100}),
        'Stk Dist Mid 26W': calculate_grade(optimum_value['Stk Dist Mid 26W'], {(optimum_value['Stk Dist Mid 26W'] - 15):10, optimum_value['Stk Dist Mid 26W']:100, (optimum_value['Stk Dist Mid 26W'] + 30):10}),
        'IV Gap Stk': 0, # TODO 
        'IV P/C': calculate_grade(optimum_value['IV P/C'], {0.8:0, 1.1:100, 1.4:0}),
        'P Margin': calculate_grade(optimum_value['P Margin'], {5:100, 10:60, 15:0}),
        'C Margin': calculate_grade(optimum_value['C Margin'], {10:100, 15:75, 20:0}),
        'Max Margin': calculate_grade(optimum_value['Max Margin'], {10:100, 15:75, 20:0}),
        'RSI': calculate_grade(optimum_value['RSI'], {30:0, 40:100, 50:100, 60:0})
        }

        optimum_weight = {
        'OPTIMUM': "Weight", 
        'Total': 0,
        'Yearly%': 0,
        'Delta Ratio': 0,
        'Delta Neutral': 0,
        'Vega Neutral': 0,
        'Day P1': optimum_grade['Day P1'] * 5,
        'Day P2': optimum_grade['Day P2'] * 5,
        'Day C3': optimum_grade['Day C3'] * 5,
        'Day C4': optimum_grade['Day C4'] * 5,
        'Stk Dist Mid 26W': optimum_grade['Stk Dist Mid 26W'] * 5,
        'IV Gap Stk': 0,
        'IV P/C': optimum_grade['IV P/C'] * 5,
        'P Margin': optimum_grade['P Margin'] * 5,
        'C Margin': optimum_grade['C Margin'] * 5,
        'Max Margin': optimum_grade['Max Margin'] * 10,
        'RSI': optimum_grade['RSI'] * 10
        }

        ds_optimum = pd.DataFrame([optimum_value, optimum_grade, optimum_weight])
        ds_optimum.set_index('OPTIMUM', inplace = True)
        ds_optimum.to_excel(writer, sheet_name = 'Bundles', startrow = currow, startcol = 0)
        currow += 5 # 4 for the ds_
        # ----------------- PROFIT DF -------------------------
        profit_value = {
        'PROFIT': "Value", 
        'Total': " - ", # TODO 
        'Cost-P': 0, #round(mark_p1 / (mark_p1 + mark_p2), 2), #Q: DIVIDED BY ZERO
        'Cost-C': 0, #round(mark_c4 / (mark_c3 + mark_c4), 2), #Q: DIVIDED BY ZERO
        'Cost-P/C': round((mark_p1 + mark_c4) / (mark_p1 + mark_p2 + mark_c3 + mark_c4), 2),
        'Theta-P1/P2': 0,
        'Theta-C3/C4': 0,
        'RTM-P': round(rtd_p * 22, 2),
        'RTM-C': round(rtd_c * 22, 2),
        'IV-P-Gap': round(iv_p2 - iv_p1 ,2),
        'IV-C-Gap': round(iv_c3 - iv_c4 ,2)
        }

        profit_grade = {
        'PROFIT': "Grade", 
        'Total': " - ", # TODO 
        'Cost-P': calculate_grade(profit_value['Cost-P'], {0:100 , 0.1:100, 0.5:0}),
        'Cost-C': calculate_grade(profit_value['Cost-P'], {0:100 , 0.1:100, 0.5:0}),
        'Cost-P/C': calculate_grade(profit_value['Cost-P/C'], {0:100 , 0.1:100, 0.5:0}),
        'Theta-P1/P2': calculate_grade(profit_value['Theta-P1/P2'], {0:20 , 1.5:20, 2.1:100, float('inf'):100}),
        'Theta-C3/C4': calculate_grade(profit_value['Theta-C3/C4'], {0:20 , 1.5:20, 2.1:100, float('inf'):100}),
        'RTM-P': -2,
        'RTM-C': -2,
        'IV-P-Gap': calculate_grade(profit_value['IV-P-Gap'], {0:0 , 0.5:0, 2.0:30, 4.0:70, 8.0:100, float('inf'):100}),
        'IV-C-Gap': calculate_grade(profit_value['IV-C-Gap'], {0:0 , 0.5:0, 2.0:30, 4.0:70, 8.0:100, float('inf'):100})
        }
                

        profit_weight = {
        'PROFIT': "Weight", 
        'Total': " - ", # TODO 
        'Cost-P': profit_grade['Cost-P'] * 10,
        'Cost-C': profit_grade['Cost-C'] * 10,
        'Cost-P/C': profit_grade['Cost-P/C'] * 10,
        'Theta-P1/P2': profit_grade['Theta-P1/P2'] * 10,
        'Theta-C3/C4': profit_grade['Theta-C3/C4'] * 10,
        'RTM-P': profit_grade['RTM-P'] * 5,
        'RTM-C': profit_grade['RTM-C'] * 5,
        'IV-P-Gap': profit_grade['IV-P-Gap'] * 10,
        'IV-C-Gap':profit_grade['IV-P-Gap'] * 10
        }            

        ds_profit = pd.DataFrame([profit_value, profit_grade, profit_weight])
        ds_profit.set_index('PROFIT', inplace = True)
        ds_profit.to_excel(writer, sheet_name = 'Bundles', startrow = currow, startcol = 0)
        currow += 5 # 4 for the ds_


        # ----------------- RISK DF -------------------------
        risk_value = {
        'RISK': "Value", 
        'Total': " - ", # TODO 
        'VIX': -2, 
        'DELTA-RISK-P1': delta_p1, 
        'DELTA-RISK-P2': delta_p2,
        'DELTA-RISK-P2/C3': 0,
        'DELTA-RISK-P1/C4': 0,
        'DELTA-RISK-C3': delta_c3,
        'DELTA-RISK-C4': delta_c4,
        'STK-PO-P1': round((((last_price - strike_p1) / days_p1) / last_price), 2),
        'STK-PO-P2': round((((last_price - strike_p2) / days_p2) / last_price), 2),
        'STK-PO-C3': round((((last_price - strike_c3) / days_c3) / last_price), 2),
        'STK-PO-C4': round((((last_price - strike_c4) / days_c4) / last_price), 2),
        'IVP/IVC': round(iv_p2 / iv_c3 ,2),
        'C3-HIGH-26W': maxi - strike_c3,
        'P2-LOW-26W': mini - strike_p2
        }

        risk_grade = {
        'RISK': "Grade", 
        'Total': " - ", # TODO 
        'VIX': calculate_grade(risk_value['VIX'], {100:0 , 100:15, 10:20, 10:math.inf}),
        'DELTA-RISK-P1': calculate_grade(risk_value['DELTA-RISK-P1'], {100:0 , 100:15 , 10:20}), #TODO
        'DELTA-RISK-P2': calculate_grade(risk_value['DELTA-RISK-P2'], {100:0 , 100:0.2 , 0:0.3 , }), #FLIP, COMPLETE
        'DELTA-RISK-P2/C3': calculate_grade(risk_value['DELTA-RISK-P2/C3'], {100:0 , 100:15, 10:20}), #TODO
        'DELTA-RISK-P1/C4': calculate_grade(risk_value['DELTA-RISK-P1/C4'], {100:0 , 100:15, 10:20}), #TODO
        'DELTA-RISK-C3': calculate_grade(risk_value['DELTA-RISK-C3'], {100:0 , 100:15, 10:20}), #TODO
        'DELTA-RISK-C4': calculate_grade(risk_value['DELTA-RISK-C4'], {100:0 , 100:15, 10:20}), #TODO
        'STK-PO-P1': 0,
        'STK-PO-P2': 0,
        'STK-PO-C3': 0,
        'STK-PO-C4': 0,
        'IVP/IVC': 0,
        'C3-HIGH-26W': 0,
        'P2-HIGH-26W': 0
        }
                

        risk_weight = {
        'RISK': "Weight", 
        'Total': " - ", # TODO 
        'VIX': 0,
        'DELTA-RISK-P1': 0,
        'DELTA-RISK-P2': 0,
        'DELTA-RISK-P2/C3': 0,
        'DELTA-RISK-P1/C4': 0,
        'DELTA-RISK-C3': 0,
        'DELTA-RISK-C4': 0,
        'STK-PO-P1': 0,
        'STK-PO-P2': 0,
        'STK-PO-C3': 0,
        'STK-PO-C4': 0,
        'IVP/IVC': 0,
        'C3-HIGH-26W': 0,
        'P2-HIGH-26W': 0
        }            

        ds_ = pd.DataFrame([risk_value, risk_grade, risk_weight])
        ds_.set_index('RISK', inplace = True)
        ds_.to_excel(writer, sheet_name = 'Bundles', startrow = currow, startcol = 0)
        currow += 7 # 4 for the ds_, 3 for spacing
        bundle_count += 1

writer.save() #save the excelt sheet

NameError: name 'pd' is not defined